In [25]:
import numpy as np
import pandas as pd 

In [26]:
tab1 = pd.read_csv("./data/table1.csv")
tab1.rename(columns={
    "شناسه‌ی عضو" : 'personID',
    "شناسه‌ی بخش" : 'unitID',
    "ضریب اهمیت در بخش":"importance_unit",
    "میزان انتظار":"expected"
},inplace=True)
tab1.head()

,personID,unitID,importance_unit,expected
0,810,1,0.82,21
1,284,1,0.73,34
2,175,1,0.83,43
3,436,1,0.85,44
4,289,1,0.71,31


In [27]:
tab1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   personID         1484 non-null   int64  
 1   unitID           1484 non-null   int64  
 2   importance_unit  1484 non-null   float64
 3   expected         1484 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 46.5 KB


In [28]:
tab2  = pd.read_csv("./data/table2.csv")
tab2.rename(columns={
  'شناسه‌ی بخش': 'unitID',
  'حقوق ساعتی':'hourly_income'
},inplace=True)
tab2.head()

,unitID,hourly_income
0,1,36000
1,2,26000
2,3,20000
3,4,24000
4,5,35000


In [29]:
tab2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   unitID         10 non-null     int64
 1   hourly_income  10 non-null     int64
dtypes: int64(2)
memory usage: 288.0 bytes


In [30]:
tab3 = pd.read_csv("./data/table3.csv")
tab3.rename(columns={
    'شناسه‌ی عضو':"personID",
    'شناسه‌ی بخش':'unitID',
    'ماه':'month',
    'ساعت کاری':'working_time'
},inplace=True)
tab3.head()

,personID,unitID,month,working_time
0,810,1,1,20
1,810,1,2,13
2,810,1,3,23
3,810,1,4,25
4,810,1,5,14


In [31]:
tab3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17808 entries, 0 to 17807
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   personID      17808 non-null  int64
 1   unitID        17808 non-null  int64
 2   month         17808 non-null  int64
 3   working_time  17808 non-null  int64
dtypes: int64(4)
memory usage: 556.6 KB


In [32]:
months = pd.DataFrame({"month": range(1, 13)})
grid = (
    tab1[["personID", "unitID", "expected"]]
    .merge(months, how="cross")  
)
grid.head()

,personID,unitID,expected,month
0,810,1,21,1
1,810,1,21,2
2,810,1,21,3
3,810,1,21,4
4,810,1,21,5


In [33]:
work = tab3[["personID", "unitID", "month", "working_time"]]
work

,personID,unitID,month,working_time
0,810,1,1,20
1,810,1,2,13
2,810,1,3,23
3,810,1,4,25
4,810,1,5,14
...,...,...,...,...
17803,578,10,8,55
17804,578,10,9,34
17805,578,10,10,21
17806,578,10,11,60


In [34]:
grid = grid.merge(work,on=["personID","unitID","month"],how='left')
grid

,personID,unitID,expected,month,working_time
0,810,1,21,1,20
1,810,1,21,2,13
2,810,1,21,3,23
3,810,1,21,4,25
4,810,1,21,5,14
...,...,...,...,...,...
17803,578,10,37,8,55
17804,578,10,37,9,34
17805,578,10,37,10,21
17806,578,10,37,11,60


In [35]:
grid.isna().sum()

personID        0
unitID          0
expected        0
month           0
working_time    0
dtype: int64

In [36]:
grid["below_expect_in_unit"] = grid["working_time"] < grid["expected"]

grid

,personID,unitID,expected,month,working_time,below_expect_in_unit
0,810,1,21,1,20,True
1,810,1,21,2,13,True
2,810,1,21,3,23,False
3,810,1,21,4,25,False
4,810,1,21,5,14,True
...,...,...,...,...,...,...
17803,578,10,37,8,55,False
17804,578,10,37,9,34,True
17805,578,10,37,10,21,True
17806,578,10,37,11,60,False


In [37]:
lost_by_member_month = (
    grid.groupby(["personID", "month"], as_index=False)["below_expect_in_unit"]
        .any()
        .rename(columns={"below_expect_in_unit": "lost_this_month"})
)
lost_by_member_month.iloc[:14,:]

,personID,month,lost_this_month
0,1,1,True
1,1,2,True
2,1,3,True
3,1,4,True
4,1,5,False
5,1,6,True
6,1,7,True
7,1,8,True
8,1,9,True
9,1,10,True


In [38]:
lost_count = (
    lost_by_member_month.groupby("personID", as_index=False)["lost_this_month"]
        .sum()
        .rename(columns={"lost_this_month": "lost_months"})
)
lost_count

,personID,lost_months
0,1,10
1,2,10
2,3,10
3,4,6
4,5,10
...,...,...
796,994,5
797,996,11
798,997,7
799,998,9


In [39]:
fired = lost_count.assign(fired=lambda d: d["lost_months"] >= 6)
fired

,personID,lost_months,fired
0,1,10,True
1,2,10,True
2,3,10,True
3,4,6,True
4,5,10,True
...,...,...,...
796,994,5,False
797,996,11,True
798,997,7,True
799,998,9,True


In [40]:
fired_list = fired.loc[fired["fired"], "personID"].tolist()
survivors_list = fired.loc[~fired["fired"], "personID"].tolist()

In [41]:
tab1_surv = tab1[tab1["personID"].isin(survivors_list)].copy()
tab1_surv

,personID,unitID,importance_unit,expected
10,67,1,0.63,29
22,844,1,0.55,30
30,908,1,0.58,24
45,63,1,0.55,20
49,423,1,0.62,16
...,...,...,...,...
1351,891,10,0.87,54
1359,128,10,0.66,29
1417,791,10,0.75,27
1462,684,10,0.74,44


In [42]:
worked_year = (
    tab3.groupby(["personID", "unitID"], as_index=False)["working_time"]
       .sum()
       .rename(columns={"worked_hours": "worked_year"})
)
worked_year

,personID,unitID,working_time
0,1,8,445
1,2,5,501
2,2,8,466
3,3,6,581
4,3,7,267
...,...,...,...
1479,996,10,222
1480,997,1,418
1481,998,4,328
1482,998,5,291


In [43]:
expected_year = (
    tab1_surv[["personID", "unitID", "expected", "importance_unit"]]
        .assign(expected_year=lambda d: 12 * d["expected"])
)
expected_year

,personID,unitID,expected,importance_unit,expected_year
10,67,1,29,0.63,348
22,844,1,30,0.55,360
30,908,1,24,0.58,288
45,63,1,20,0.55,240
49,423,1,16,0.62,192
...,...,...,...,...,...
1351,891,10,54,0.87,648
1359,128,10,29,0.66,348
1417,791,10,27,0.75,324
1462,684,10,44,0.74,528


In [44]:

perf = expected_year.merge(worked_year, on=["personID", "unitID"], how="left")

perf["working_time"] = perf["working_time"].fillna(0.0)

perf["ratio"] = perf["working_time"] / perf["expected_year"]

perf.loc[perf["expected_year"] <= 0, "ratio"] = 0.0
perf

,personID,unitID,expected,importance_unit,expected_year,working_time,ratio
0,67,1,29,0.63,348,385,1.106322
1,844,1,30,0.55,360,360,1.000000
2,908,1,24,0.58,288,311,1.079861
3,63,1,20,0.55,240,261,1.087500
4,423,1,16,0.62,192,232,1.208333
...,...,...,...,...,...,...,...
113,891,10,54,0.87,648,669,1.032407
114,128,10,29,0.66,348,353,1.014368
115,791,10,27,0.75,324,320,0.987654
116,684,10,44,0.74,528,544,1.030303


In [45]:
def importance_bonus(r):
    if (r > 1.0) and (r <= 1.25):
        return 0.25
    elif (r > 1.25) and (r <= 1.5):
        return 0.50
    elif r > 1.5:
        return 1.00
    return 0.0

In [ ]:

perf["importance_bonus"] = perf["ratio"].apply(importance_bonus)
perf["importance_unit_new"] = perf["importance_unit"] + perf["importance_bonus"]


tab2_fixed = tab2.copy()
if "hourly_income" in tab2_fixed.columns and "hourly_wage" not in tab2_fixed.columns:
    tab2_fixed = tab2_fixed.rename(columns={"hourly_income": "hourly_wage"})

perf = perf.merge(tab2_fixed[["unitID", "hourly_wage"]], on="unitID", how="left")


perf["hourly_wage"] = perf["hourly_wage"].fillna(0.0)
perf = perf.rename(columns={"working_time": "worked_year"})


perf["new_wage"] = np.floor(perf["importance_unit_new"] * perf["hourly_wage"]).astype(int)

# 6) پیش‌نمایش سریع
print(perf[[
    "personID","unitID","importance_unit","worked_year","expected_year",
    "ratio","importance_unit_new","hourly_wage","new_wage"
]].head(10).to_string(index=False))

 personID  unitID  importance_unit  worked_year  expected_year    ratio  importance_unit_new  hourly_wage  new_wage
       67       1             0.63          385            348 1.106322                 0.88        36000     31680
      844       1             0.55          360            360 1.000000                 0.55        36000     19800
      908       1             0.58          311            288 1.079861                 0.83        36000     29880
       63       1             0.55          261            240 1.087500                 0.80        36000     28800
      423       1             0.62          232            192 1.208333                 0.87        36000     31320
      230       1             0.76          410            420 0.976190                 0.76        36000     27360
      663       1             0.70          470            432 1.087963                 0.95        36000     34200
      207       1             0.77          588            540 1.088889 

In [ ]:
tab2_fixed = tab2.copy()
if "hourly_wage" not in tab2_fixed.columns and "hourly_income" in tab2_fixed.columns:
    tab2_fixed = tab2_fixed.rename(columns={"hourly_income": "hourly_wage"})

perf = perf.merge(
    tab2_fixed[["unitID", "hourly_wage"]],
    on="unitID",
    how="left",
    suffixes=("", "_tab2")
)

if "hourly_wage_tab2" in perf.columns and "hourly_wage" in perf.columns:

    perf["hourly_wage"] = perf["hourly_wage"].fillna(perf["hourly_wage_tab2"])
    perf = perf.drop(columns=["hourly_wage_tab2"])
elif "hourly_wage_tab2" in perf.columns:
    perf = perf.rename(columns={"hourly_wage_tab2": "hourly_wage"})


perf["hourly_wage"] = perf["hourly_wage"].fillna(0.0)
perf

,personID,unitID,expected,importance_unit,expected_year,worked_year,ratio,importance_bonus,importance_unit_new,hourly_wage,new_wage
0,67,1,29,0.63,348,385,1.106322,0.25,0.88,36000,31680
1,844,1,30,0.55,360,360,1.000000,0.00,0.55,36000,19800
2,908,1,24,0.58,288,311,1.079861,0.25,0.83,36000,29880
3,63,1,20,0.55,240,261,1.087500,0.25,0.80,36000,28800
4,423,1,16,0.62,192,232,1.208333,0.25,0.87,36000,31320
...,...,...,...,...,...,...,...,...,...,...,...
113,891,10,54,0.87,648,669,1.032407,0.25,1.12,33000,36960
114,128,10,29,0.66,348,353,1.014368,0.25,0.91,33000,30030
115,791,10,27,0.75,324,320,0.987654,0.00,0.75,33000,24750
116,684,10,44,0.74,528,544,1.030303,0.25,0.99,33000,32670


In [49]:
final_df = perf[["personID", "unitID", "new_wage"]].copy()
final_df = final_df.rename(columns={
    "personID": "0",
    "unitID": "1",
    "new_wage": "2"
})


final_df.to_csv("./data/submission.csv", index=False)

print(final_df.head())


     0  1      2
0   67  1  31680
1  844  1  19800
2  908  1  29880
3   63  1  28800
4  423  1  31320
